# Coche d'un chèque dans "Remise en banque"

In [ ]:
port = "COM4"
import serial
import os


def read_code():
    code = ""
    car = ''
    with serial.Serial(port) as ser:
        while car != b'\r':
            car = ser.read()
            code = code + car.decode('utf-8')
        return code


def read_and_select_code():
    code = read_code().strip()
    if code == 'STOP':
        return code
    code_sans_annee=code[:-4]
    code_final=code_sans_annee[-9:]
    print(f"{code} → {code_final}")
    os.system(f"select_cheque.exe {code_final}")
    return code_final


while read_and_select_code() != 'STOP':
    pass


# Saisie d'un cheque ANCV
Scanner le code du montant (cf plus bas).

Se placer sur la transaction, liste des paiement déjà ouvert.

Scanner le chèque, attendre sa saisie. Rescanner le chèque pour valider ou le code "STOP" (cf plus bas). Si STOP → il faut nettoyer la saisie (supprimer le chèque en cours de saisie, et fermer la saisie en cours).

| ![Stop](./stop.png) | ****************** ![10€](./M10.png) | ****************** ![20€](./M20.png) | ****************** ![50€](./M50.png) |
|--|--|--|--|
|Stop|10€|20€|50€|

In [ ]:
port = "COM4"
montant = 10
import serial
import os

def read_code():
    code = ""
    car = ''
    with serial.Serial(port) as ser:
        while car != b'\r':
            car = ser.read()
            code = code + car.decode('utf-8')
        return code


def read_and_select_code():
    code = read_code().strip()
    if code == 'STOP':
        return code
    if code.startswith('M'):
        new_montant = int(code[1:])
        globals()['montant'] = new_montant
        print(f"Montant: {new_montant} €")
        return code
    code_sans_annee=code[:-4]
    code_final=code_sans_annee[-9:]
    print(f"{code_final}", end=None)
    os.system(f"auto_saisie_ancv.exe {montant} {code_final}")
    # Validation
    code_validation = read_code().strip()
    if code_validation == 'STOP':
        return code_validation
    if code == code_validation:
        code_sans_annee=code_validation[:-4]
        code_final=code_sans_annee[-9:]
        print(f"✅")
        os.system(f"auto_saisie_ancv.exe {montant} {code_final} --confirm")
    return code_final



while read_and_select_code() != 'STOP':
    pass

In [ ]:
import qrcode
map={"continuous":"S_CMD_020E", "auto-sensing":"S_CMD_020F","trigger":"S_CMD_MT00"}
for i, k in map.items():
    img = qrcode.make(k)
    img.save(f"{i}.png")

# Code barres spéciaux de configuration du lecteur de code barre

|Continuous|Anto-sensing|Trigger|
|--|--|--|
| ![Continuous](./continuous.png) | ************************ ![Continuous](./auto-sensing.png) | ************************ ![Continuous](./trigger.png) |

# Saisie groupée

In [ ]:
import qrcode
map={"stop":"STOP", "reset":"XRESTE", "go":"XGO", "retry":"XRETRY", "M10":"M10","M20":"M20","M50":"M50"}
for i, k in map.items():
    img = qrcode.make(k)
    img.save(f"{i}.png")

| ![Stop](./stop.png) | ****************** ![10€](./M10.png) | ****************** ![20€](./M20.png) | ****************** ![50€](./M50.png) | 
|--|--|--|--|
|Stop|10€|20€|50€|
|******************|******************|******************|******************|******************|
||****************** ![GO](./go.png) |****************** ![RESET](./reset.png) | ****************** ![RETRY](./retry.png) |
||GO|RESET|RETRY|

In [ ]:
port = "COM4"
montant = 10
import serial
import os

def read_code():
    code = ""
    car = ''
    with serial.Serial(port) as ser:
        while car != b'\r':
            car = ser.read()
            code = code + car.decode('utf-8')
        return code

def write(code,amount):
    # print(f"Cheque {code} {amount}€")
    os.system(f"auto_saisie_ancv.exe {amount} {code}")
    # code_validation = read_code().strip()
    # if (code_validation == 'STOP'):
    #     # print("STOP")
    #     return code_validation
    # elif (code_validation == "XGO"):
    #     # print("OK")
    #     os.system(f"auto_saisie_ancv.exe {amount} {code} --confirm")
    #     # code_validation = read_code().strip()
    #     # if (code_validation == 'STOP'):
    #     #     print("OK")
    #     #     return code_validation
    #     # else:
    #     #     return code_validation
    # else:
    #     write(code,amount)
codes={}
def read_and_select_code_grouped():
    if globals()['codes']==None:
        globals()['codes']={}
    code = read_code().strip()
    if code == 'STOP':
        return code
    if code.startswith('M'):
        new_montant = int(code[1:])
        globals()['montant'] = new_montant
        print(f"  --> Montant: {new_montant} €")
        return code
    elif code.startswith('X'):
        if code=="XRESET" or code=='XRESTE':
            globals()['codes']={}
            print(f"Reset {len(globals()['codes'])} chèque{'s' if len(globals()['codes']) > 1 else ''}")
        elif code=="XGO":
            print(f"Go pour {len(globals()['codes'])} chèque{'s' if len(globals()['codes']) > 1 else ''}")
            for code, amount in globals()['codes'].items():
                ret = write(code, amount)
                if ret == "STOP":
                    return ret
            print("Done")
            globals()['codes'] = {}
    else:
        code_sans_annee=code[:-4]
        code=code_sans_annee[-9:]
        globals()['codes'][code] = montant
        print(f"{code} {montant}€ Total : {sum(globals()['codes'].values())}€ {len(globals()['codes'])} chèque{'s' if len(globals()['codes']) > 1 else ''}")
        #os.system(f"auto_saisie_ancv.exe {montant} {code}")
        # Validation
        #os.system(f"auto_saisie_ancv.exe {montant} {code} --confirm")
    return code


while read_and_select_code_grouped() != 'STOP':
    pass